In [1]:
# import internal .py modules
import file_path_management as fpath
import public_library as plib

2023-08-08 14:57:23 Didis-MacBook-Pro.local metapub.config[39330] WARNING NCBI_API_KEY was not set.


10.1016/j.conb.2007.07.003
17707635


In [2]:
# import packages
import csv
import pandas as pd
import PyPDF2
import requests
import time
import os
import random
from requests.auth import HTTPProxyAuth
from bs4 import BeautifulSoup
import numpy as np
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import TimeoutException, WebDriverException, NoSuchElementException
import os
import re
from lxml import etree

In [4]:
# merge all search results, fill in elements missed, remove duplciations based on identifiers in the potential related literature
def merge_everything(input, output):
    # combine all results
    df = pd.DataFrame()
    for search_result in input:
        df_single = pd.read_csv(search_result, header=None, sep = ",")
        df = df.append(df_single, ignore_index=True, sort=False)
    df.columns = ["DOI", "PMID", "PMCID", "Title", "full_text_url", "full_text_source", "pdf_url", "pdf_source"]
    print(len(df))
    
    # fill in elements that are missing
    # DOI -> PMID
    # PMID -> DOI
    # PMID -> PMCID
    # PMCID -> PMID
    for ind in df.index:
        # doi
        if df["DOI"][ind] == df["DOI"][ind]:
            doi = str(df["DOI"][ind]).strip()
        elif df["PMID"][ind] == df["PMID"][ind]:
            doi = plib.pmid2doi(df["PMID"][ind])
        elif ():


            pmid = df["PMID"][ind]
            url = "https://pubmed.ncbi.nlm.nih.gov/" + pmid + "/"
            print(url)
            response = requests.get(url, headers = plib.headers)
            if response.status_code != 200:
                raise Exception("Error when request webpages!")
            soup = BeautifulSoup(response.content, "lxml")
            l = soup.find_all("a", {"class: id-link"}, {"data-ga-action": "DOI"})
            if(len(l) != 0):
                # print(l[0].get_text().strip())
                df.at[ind, "DOI"] = l[0].get_text().strip()
            else:
                df.at[ind, "DOI"] = np.nan
        # pmid
        if (df["PMID"][ind] == df["PMID"][ind]):
            pmid = df["PMID"][ind]
            url = "https://pubmed.ncbi.nlm.nih.gov/" + pmid + "/"
            print(url)
            response = requests.get(url, headers = plib.headers)
            if response.status_code != 200:
                raise Exception("Error when request webpages!")
            soup = BeautifulSoup(response.content, "lxml")
            l = soup.find_all("a", {"class: id-link"}, {"data-ga-action": "DOI"})
            if(len(l) != 0):
                # print(l[0].get_text().strip())
                df.at[ind, "DOI"] = l[0].get_text().strip()
            else:
                df.at[ind, "DOI"] = np.nan
        # pmcid
        # title

        # full_text_link
        # pdf_link
        pdf_link = df["PMID"][ind]

    # remove all duplicates
    df = df.drop_duplicates(subset=['DOI'])
    df = df.drop_duplicates(subset=['PMID'])
    df = df.drop_duplicates(subset=['PMCID'])
    print(len(df))
    # plib.clear_file(fpath.poten_litera_csv)
    # df.csv(fpath.poten_litera_csv, idnex = None)
    print("Duplication in the potential related literature removed.")


    print("Found " + len(df) + " potential related literature in total.")

    # write the results into our output file
    df.column = ["DOI", "PMID", "PMCID", "Title", "Full_text_link", "PDF_link"]
    df.to_csv(output, header=True, index=False)
# --------------------start of test code--------------------
# gos = fpath.poten_litera_gs_processed_step2
# wos = fpath.poten_litera_wos_processed
# pubmed = fpath.poten_litera_pubmed_processed
# eupmc = fpath.poten_litera_eupmc_processed
# input = [gos, wos, pubmed, eupmc]
# output = fpath.poten_litera
# merge_everything(input, output)
# ---------------------end of test code---------------------

In [5]:
# # merge all search results

# # identifier = ["DOI", "PMID", "PMCID"]
# file_path = fpath.poten_litera
# merge_everything(file_path)